In [1]:
import torch.nn as nn

from datasets import *
from evaluation import *
from models import *
from pruning.unstructured import *
from pruning.structured import *
from pyJoules.device.nvidia_device import NvidiaGPUDomain

Importing /raid/home/prabhasreddy/yaswanth/smart-pruner/pruning/unstructured/L1norm.py
Importing /raid/home/prabhasreddy/yaswanth/smart-pruner/pruning/Train.py
Importing /raid/home/prabhasreddy/yaswanth/smart-pruner/pruning/unstructured/random.py
Importing /raid/home/prabhasreddy/yaswanth/smart-pruner/pruning/structured/decay_pruning.py
Importing /raid/home/prabhasreddy/yaswanth/smart-pruner/pruning/structured/two_to_four.py
Importing /raid/home/prabhasreddy/yaswanth/smart-pruner/pruning/structured/Thinet.py
Importing /raid/home/prabhasreddy/yaswanth/smart-pruner/pruning/structured/Global_pruning.py
Importing /raid/home/prabhasreddy/yaswanth/smart-pruner/pruning/structured/Ln_structured.py


/raid/home/prabhasreddy/yaswanth/smart-pruner/venv/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# dataset = MNIST()
dataset = CIFAR10()
train_loader, test_loader = dataset.get_dataloader()

Files already downloaded and verified
Files already downloaded and verified


In [3]:
#train model and return it
# model = Lenet()
model = ResNet50_Cifar10()
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
model = Trainer(model, 5, train_loader, criterion, optimizer)
original_model = model.train()
#print("Accuracy of original model : ", accuracy(original_model, test_loader))

 20%|██        | 1/5 [00:16<01:05, 16.43s/it]

Epoch: 1 | train_loss: 6.4954 | 


 40%|████      | 2/5 [00:31<00:47, 15.69s/it]

Epoch: 2 | train_loss: 6.3084 | 


 60%|██████    | 3/5 [00:46<00:30, 15.26s/it]

Epoch: 3 | train_loss: 6.1653 | 


 80%|████████  | 4/5 [01:00<00:14, 14.96s/it]

Epoch: 4 | train_loss: 6.1076 | 


100%|██████████| 5/5 [01:14<00:00, 14.95s/it]

Epoch: 5 | train_loss: 5.9600 | 


In [20]:
#for UnstructuredL1normPrune

# model = Lenet()
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(original_model.parameters(), lr=0.001)

# trainer = UnstructuredL1normPrune(original_model, 5, train_loader, criterion, optimizer, 0.9)
trainer = GlobalPrune(original_model, 5, train_loader, criterion, optimizer, 0.5)

#train and prune the model and return both the original and pruned model
#original_model, pruned_model = trainer.train_prune_retrain()
pruned_model = trainer.prune_model()

In [6]:
# %%bash --bg 
# while true; do nvidia-smi -i 0 --format=csv,noheader,nounits --query-gpu=power.draw >> unpruned_output.txt; sleep 0.1; done

In [7]:
print("Accuracy of original model : ", accuracy(original_model, test_loader))

# with open('unpruned_output.txt', 'a') as f:
#     f.write("\energy computation for original model completed.\n")

begin timestamp : 1712778846.2185016; tag : accuracy; duration : 1.787135362625122; nvidia_gpu_0 : 101351
Accuracy of original model :  [66.08]


In [7]:
# %%bash
# pgrep -f "watch -n 1 'nvidia-smi -i 0 --format=csv,noheader,nounits --query-gpu=power.draw >> unpruned_output.txt'" | xargs -r kill

In [8]:
#for pruned 

In [9]:
# %%bash --bg 
# while true; do nvidia-smi -i 0 --format=csv,noheader,nounits --query-gpu=power.draw >> pruned_output.txt; sleep 0.1; done

In [15]:
print("Accuracy of pruned model : ", accuracy(pruned_model, test_loader))

# with open('pruned_output.txt', 'a') as f:
#     f.write("\energy computation for pruned model completed.\n")

begin timestamp : 1712778973.0649247; tag : accuracy; duration : 1.7264270782470703; nvidia_gpu_0 : 97941
Accuracy of pruned model :  [36.83]


In [11]:
# %%bash
# pgrep -f "watch -n 1 'nvidia-smi -i 0 --format=csv,noheader,nounits --query-gpu=power.draw >> pruned_output.txt'" | xargs -r kill

In [12]:
#done

In [28]:
# with EnergyContext(domains=[NvidiaGPUDomain(0)]):
#     accuracy(original_model, test_loader)
temp = accuracy(original_model, test_loader)
accuracy(pruned_model, test_loader)

begin timestamp : 1712779332.8319275; tag : accuracy; duration : 1.6843435764312744; nvidia_gpu_0 : 95537
begin timestamp : 1712779334.518447; tag : accuracy; duration : 1.625730276107788; nvidia_gpu_0 : 92260


array([65.9])

In [18]:
print(temp)

[66.08]
